# Term Paper

### Part 1

In [34]:
import pandas as pd
import glob as glob

In [35]:
#Defining a function to compute and print dataset statistics
def compute_statistics():
    num_individuals = sce_df["userid"].nunique() # Number of Unique userids (individuals)
    num_observations = len(sce_df)               # Total number of observations
    num_waves = sce_df["wid"].nunique()          # Number of Unique survey waves
    first_date = sce_df["date"].min()            # Earliest survey date
    last_date = sce_df["date"].max() # Latest survey date

    # Output the statistics
    print(f"Unique individuals: {num_individuals}") 
    print(f"Total observations: {num_observations}")
    print(f"Unique survey waves: {num_waves}")
    print(f"Date range: {first_date.date()} to {last_date.date()}")

In [39]:
files=glob.glob(f'data/SCE-*-20*.csv')

df_list=[]

waves_number=0

#Loop to read in and append files
for file in files:
    df=pd.read_csv(file,sep=";",parse_dates=['date'])
    df_list.append(df)

#Concatenate all dataframes into a single dataframe
sce_df=pd.concat(df_list,ignore_index=True)

compute_statistics()

Unique individuals: 23369
Total observations: 176101
Unique survey waves: 139
Date range: 2013-06-01 to 2024-12-31


### Part 2

In [40]:
# Forward-fill numeracy variables for each individual
numeracy_cols = [
    'num_lit_q1_correct', 'num_lit_q2_correct', 'num_lit_q3_correct',
    'num_lit_q5_correct', 'num_lit_q6_correct', 'num_lit_q8_correct', 'num_lit_q9_correct'
]

# Replace blank strings with NaN (if your CSV uses empty fields)
sce_df[numeracy_cols] = sce_df[numeracy_cols].replace("", pd.NA)


# Group by userid, then forward-fill within each person
sce_df[numeracy_cols] = sce_df.groupby('userid')[numeracy_cols].ffill()

# Drop all rows that still have NaN
numeracy_cols.extend(['female','educ','age','inflation','house_price_change','prob_stocks_up'])
sce_df.dropna(subset=numeracy_cols,inplace=True)

#Eliminate 0.1th percentile and 99.9th percentile of answers
numeracy_cols = [
    'num_lit_q1_correct', 'num_lit_q2_correct', 'num_lit_q3_correct',
    'num_lit_q5_correct', 'num_lit_q6_correct', 'num_lit_q8_correct', 'num_lit_q9_correct'
]
lower_bounds=sce_df[numeracy_cols].quantile(0.001)
upper_bounds=sce_df[numeracy_cols].quantile(0.999)
for col in numeracy_cols:
    sce_df=sce_df[(sce_df[col]>=lower_bounds[col]) & (sce_df[col]<=upper_bounds[col])]

#Creating a column to determine whether the individual has a bachelor's degree or higher
sce_df['college']=(sce_df['educ']>3).astype(int)

#Compute the total number of correct numeracy responces
sce_df['numeracy_score']=sce_df[numeracy_cols].sum(axis=1)

#Compute the average numeracy score
percentage=sce_df['numeracy_score'].value_counts(normalize=True)*100
percentage.sort_index(inplace=True)
print(f"Numeracy Score Distribution (%): {percentage}")

#Creating a column to determine if the individual has a higher numeracy score than the median
median_score = sce_df['numeracy_score'].median()
sce_df['num_lit_high'] = (sce_df['numeracy_score'] > median_score).astype(int)

#Calling the function to compute and print updated dataset statistics
compute_statistics()

Numeracy Score Distribution (%): numeracy_score
0.0     0.111573
1.0     0.659177
2.0     2.001898
3.0     5.215707
4.0     9.579871
5.0    16.597415
6.0    27.303273
7.0    38.531086
Name: proportion, dtype: float64
Unique individuals: 17701
Total observations: 77976
Unique survey waves: 117
Date range: 2015-04-02 to 2024-12-31
